In [1]:
import math
import torch
from torch import nn
from torch import optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pathlib import Path
import pandas as pd

import ipywidgets as widgets
from ipywidgets import GridspecLayout, Button, Layout, Dropdown, BoundedFloatText, VBox, Button, Layout

MODEL_PATH = Path.cwd()/'Models'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
#import this or somehow avoid explicitely writing it here
class NN(nn.Module):
    def __init__(self, layers=[1000,500,250], ps=0.35, in_features=20, y_range=(20, 90)):
        super(NN, self).__init__()
        self.y_range = y_range
        self.layers = layers
        self.ps = ps
        layers = [in_features] + layers
        layers = list(zip(layers, layers[1:]))
        
        l = []
        for layer in layers:
            l.append(nn.Linear(*layer))
            #TODO: play with negative slope koef. of LeakyReLU
            l.append(nn.LeakyReLU())
            l.append(nn.Dropout(ps))
        l.append(nn.Linear(layers[-1][1], 1))

        self.arch = nn.Sequential(*l)
        
    def forward(self, x):
        x = self.arch(x)
        x = (self.y_range[1]-self.y_range[0]) * torch.sigmoid(x) + self.y_range[0]
        return x
    
    def __repr__(self):
        return "Linear -> LeakyReLU -> Dropout\nlayers: {}\nps: {}\n".format(self.layers, self.ps)

In [3]:
def predict(biomarkers):
    path = Path.cwd()/'Models'/'adm_state_dict'
    lm = NN()
    lm.load_state_dict(torch.load(path))
    lm.eval()
    biomarkers = torch.Tensor(biomarkers)
    return(lm(biomarkers)-27)

In [4]:
def change_x(*args):
    sex_box.index = None
    sex_box.value = "Male"
    
    for i in range(len(MALE29)+1):
        grid[i+2, 2].value = MALE29[i]

In [5]:
#TODO: Refactor with recursion
def example_change(change):
    def change_values(age):
        sex_box.index = None
        sex_box.value = "Male" if samples[age][0] == 1 else "Female"
        for i in range(len(samples[age])):
            grid[i+2, 2].value = samples[age][i+1]
        
    if change['type'] == 'change' and change['name'] == 'value':
        if change['new'] == '-':
            sex_box.index = None
            sex_box.value = "-"
            for i in range(len(list(samples.values())[0])):
                grid[i+2, 2].value = 0
        elif change['new'] == 'Male 69 y/o':
            change_values(69)
        elif change['new'] == 'Male 40 y/o':
            change_values(40)
        elif change['new'] == 'Male 29 y/o':
            change_values(29)


In [6]:
def submit_clicked(b):
    grid[-1,2].button_style="primary"
    markers = [grid[i+2, 2].value for i in range(len(biomarkers))]
    if sex_box.value=='-':
        grid[-1,2].description="Please enter your Sex"
        age.style.button_color = 'orange'
    elif 0 in markers:
        grid[-1,2].description="Please enter missing biomarkers"
        age.style.button_color = 'orange'
    else:
        markers.insert(0, int(sex_box.value=='Male'))
        grid[-1,2].description=f"You are {round(predict(markers).data.item())} years old"
        age.style.button_color = 'paleturquoise'

In [7]:
warn_text="""Age predictor needs all blood marker values and your Sex to make
a prediction. If you are missing values and filling them in, this might have
a dramatic impact on the results. However your entered Height, Weight, and
Smoking Status are not required for estimating your biological age and only
used for record keeping."""
title = "Deep Biomarkers Of Human Aging: How Old By a Basic Blood Test"
subtitle = """This is a deep-learned predictor of your age made with a DNN
trained on tens of thousands anonymized human blood tests. <br>Enter your
data below and the model will guess your age."""

In [8]:
asterics_text = [
    "* This should be in your clinical biochemistry blood test results.",
    """** Required parameter for minimal prediction accuracy of 70% within
    10 year frame.""",
    """*** We can not show you reference values before knowing your age
    apriori, so this is only a reference metric."""]

In [9]:
a="""Albumin**                        [                    ] 35 - 52 g/l
Glucose**                        [                    ] 3.9 - 5.8 mmole/l
Urea**(BUN)                      [                    ] 2.5 - 6.4 mmole/l
Cholesterol**                    [                    ] 3.37 - 5.96 mmole/l
Protein total**                  [                    ] 64 - 83 g/l
Sodium**                         [                    ] 136 - 146 mmole/l
Creatinine**                     [                    ] 53 - 97 mmole/l
Hemoglobin**                     [                    ] 11.7 - 15.5 g/dl
Bilirubin total                  [                    ] 1.7 - 21 mcmole/l
Triglycerides                    [                    ] 0.68 - 6 mmole/l
HDL Cholesterol                  [                    ] < 3.3 mmole/l
LDL cholesterol (by Friedewald)  [                    ] 1.81- 4.04 mmole/l
Calcium                          [                    ] 2.15 - 2.65 mmole/l
Potassium                        [                    ] 3.4 - 5.1 mmole/l
Hematocrit                       [                    ] 37 - 50 %
MCHC                             [                    ] 31.5 - 35.7 g/dL
MCV                              [                    ] 82 - 95 fl
Platelets                        [                    ] 150 - 450 10^3 /mcl
Erythrocytes (RBC)               [                    ] 3.5 - 5.5 10^6 /mcl"""

lines = a.splitlines()
lines = [l.split('[') for l in lines]
lines = [(b.strip(), m.strip()) for b, m in lines]

biomarkers = [l[0] for l in lines]

measures = [l[1][2:] for l in lines]
measures[4] = '6.4 - 8.3 g/dl'

biomarkers = [(i+2, biomarkers[i], measures[i]) for i in range(len(biomarkers))]

biomarkers

[(2, 'Albumin**', '35 - 52 g/l'),
 (3, 'Glucose**', '3.9 - 5.8 mmole/l'),
 (4, 'Urea**(BUN)', '2.5 - 6.4 mmole/l'),
 (5, 'Cholesterol**', '3.37 - 5.96 mmole/l'),
 (6, 'Protein total**', '6.4 - 8.3 g/dl'),
 (7, 'Sodium**', '136 - 146 mmole/l'),
 (8, 'Creatinine**', '53 - 97 mmole/l'),
 (9, 'Hemoglobin**', '11.7 - 15.5 g/dl'),
 (10, 'Bilirubin total', '1.7 - 21 mcmole/l'),
 (11, 'Triglycerides', '0.68 - 6 mmole/l'),
 (12, 'HDL Cholesterol', '< 3.3 mmole/l'),
 (13, 'LDL cholesterol (by Friedewald)', '1.81- 4.04 mmole/l'),
 (14, 'Calcium', '2.15 - 2.65 mmole/l'),
 (15, 'Potassium', '3.4 - 5.1 mmole/l'),
 (16, 'Hematocrit', '37 - 50 %'),
 (17, 'MCHC', '31.5 - 35.7 g/dL'),
 (18, 'MCV', '82 - 95 fl'),
 (19, 'Platelets', '150 - 450 10^3 /mcl'),
 (20, 'Erythrocytes (RBC)', '3.5 - 5.5 10^6 /mcl')]

In [23]:
samples = [list(pd.read_csv('Data/samples_from_aging.csv').iloc[i]) 
           for i in range(len(pd.read_csv('Data/samples_from_aging.csv')))]

samples = {sample[-1]:sample[:-1] for sample in samples}

In [12]:
height_box = widgets.Text(
    placeholder='Enter your height (Optional)',
    description='Height:',
)

weight_box = widgets.Text(
    placeholder='Enter your weight (Optional)',
    description='Weight:',
)

In [12]:
sex_box = widgets.Dropdown(options=["-","Male","Female"], description="Sex:")
smoke_box = widgets.Dropdown(options=["-","No","Yes"], description="Smoke:")

In [12]:
load_box = widgets.Dropdown(
    value='-',
    options=['-']+['Male '+str(int(x))+' y/o' for x in list(samples.keys())],
    description=" "
)

load_box.observe(example_change)

In [15]:
load_label = widgets.HTML(value = '<center><h3>Load an Example</h3></center>')
load_box = VBox([load_label, load_box])

In [16]:
header_grid = widgets.GridspecLayout(4, 5, width='auto')

warning = widgets.HTML(value = f"<center><h4><font color='orange'>{warn_text}</h4></center>")
title = widgets.HTML(value = f'<center><h1 style="font-family:verdana"><font color="paleturquoise">{title}</h1><hr><h2 style="font-family:verdana">{subtitle}</h2></center>')

header_grid[0:1,1:-1]=title
input_row = 2

header_grid[input_row,1] = height_box
header_grid[input_row,2] = weight_box
header_grid[input_row:input_row+1,3] = load_box

header_grid[input_row+1,2] = smoke_box
header_grid[input_row+1,1] = sex_box

header_grid

GridspecLayout(children=(HTML(value='<center><h1 style="font-family:verdana"><font color="paleturquoise">Deep …

In [17]:
asterics = widgets.HTML(value = f"<p><font color='orange'>{asterics_text[0]}</p><p><font color='orange'>{asterics_text[1]}</p><p><font color='orange'>{asterics_text[2]}</h4>")

In [18]:
grid = widgets.GridspecLayout(len(biomarkers)+7, 5, height='1100px', width='auto')
h = '28px'

grid[0,1:-1] = warning

for x in biomarkers:
        grid[x[0], 1] = Button(description=x[1], tooltip=x[1], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        grid[x[0], 3] = Button(description=x[2], tooltip=x[2], button_style="info", disabled=True, layout=Layout(height=h, width='auto'))
        
        grid[x[0], 2] = BoundedFloatText(
        min=0,
        max=1000,
        layout=Layout(width='auto', height=h),
        value=None,
        step=0.01,
        #description=' ',
        disabled=False
    )
        
grid[1,1] = Button(description="Blood Marker*", tooltip="Blood Marker*", button_style='primary', disabled=True, layout=Layout(height='auto', width='auto'))
grid[1,2] = Button(description="Your Value", tooltip="Your Value", button_style="primary", disabled=True, layout=Layout(height='40px', width='auto'))
grid[1,3] = Button(description="Units and Sample Metric***", tooltip="Units and Sample Metric***", button_style="primary", disabled=True, layout=Layout(height='auto', width='auto'))
        
grid[-5,1:] = asterics
submit = Button(description="Submit",tooltip="Predict your age with entered biomarkers", button_style="primary", layout=Layout(height='30px', width='auto'))
age = Button(layout=Layout(height='30px', width='auto'))
age.style.button_color = '#111111'

submit.on_click(submit_clicked)
grid[-2,2] = submit
grid[-1,2] = age

grid

GridspecLayout(children=(HTML(value="<center><h4><font color='orange'>Age predictor needs all blood marker val…

In [ ]:
footer_html = {
    'gap':'<br><br><br><br><br><br><hr>',

    'sjsu':"""Made by Dmytro Mishchenko as a project for the
    <a href='https://pytorch2021.devpost.com/'><u>PyTorch Hackathon
    </u> </a> with the help of a <a href='https://mlatsjsu.club'>
    <u>Machine Learning Club</u></a> at San Jose State University.""",

    'github':"""This project is open-source and is covered by the MIT License,
    for more details or if you want to contribure here is the
    <a href='https://github.com/mdmittriy/AgingClock'><u>
    GitHub repository</u></a>.""",

    'demo':"""You can take a look at the demo of this project <a
    href='https://www.youtube.com/watch?v=SCxRiXIG13Q'><u>here</u></a>. And
    notice that this work is heavily inspired by
    <a href='http://www.aging.ai'><u>aging.ai</u></a>"""}

widgets.HTML(value="<center>{}<br>{}<br>{}<br>{}</center>".format(
    footer_html['gap'],footer_html['sjsu'],
    footer_html['github'],footer_html['demo']))